In [1]:
from iwnlp.iwnlp_wrapper import IWNLPWrapper
import operator

In [2]:
lemmatizer = IWNLPWrapper(lemmatizer_path='C:/Users/1/Desktop/thesis/IWNLP.Lemmatizer_20170501.json')

In [3]:
def capitalize(word):
    if len(word) > 1:
        word = word[0].upper() + word[1:]
    return word

In [4]:
def get_lemma(word):
    lemma = lemmatizer.lemmatize_plain(word)
    if lemma is None:
        return capitalize(word)
    else:
        return capitalize(lemma[0])

In [5]:
file_words = open('C:/Users/1/Desktop/thesis/wortliste.txt', 'r', encoding='utf-8')
lines_words = file_words.readlines()

In [6]:
dictionary = []
for line in lines_words:
    line_new = line.split(' ')
    dictionary.append(capitalize(line_new[0]))

In [7]:
def in_dic(word):
    if word in dictionary:
        return True

In [8]:
def is_part(word):
    if len(word) > 1 and in_dic(word) is True:
        return True
    if (word.endswith('n') or word.endswith('e') or word.endswith('s') or word.endswith('-')) and in_dic(word[:-1]):
        return True
    if (word.endswith('en') or word.endswith('er') or word.endswith('es')) and in_dic(word[:-2]):
        return True

In [9]:
def choose_split(arr):
    lens = []
    for item in arr:
        var_len = len(item[0] + item[1])
        lens.append(var_len)
    index, value = max(enumerate(lens), key=operator.itemgetter(1))
    return arr[index]

In [10]:
def split(word):
    max_ind = len(word)
    splits = []
    
    for ind, char in enumerate(word):
        left_compound = word[0:max_ind-ind]
        right_compound = word[max_ind-ind:max_ind]
        
        if is_part(left_compound) and len(left_compound) != len(word):
            right_compound_upper = capitalize(right_compound)
            
            if is_part(right_compound_upper):
                if not in_dic(left_compound) and in_dic(left_compound[:-1]):
                    splits.append([left_compound[:-1], right_compound_upper])
                if not in_dic(left_compound) and in_dic(left_compound[:-2]):
                    splits.append([left_compound[:-2], right_compound_upper])
                if in_dic(left_compound):
                    splits.append([left_compound, right_compound_upper])
                 
    if len(splits) > 1:
        return choose_split(splits)
    if len(splits) == 1:
        return splits[0]
    if splits == []:
        return None

In [11]:
def split3(word):
    max_ind = len(word)
    
    for ind1 in range(max_ind):
        for ind2 in range(ind1, max_ind):
            left = word[:ind1]
            middle = word[ind1:ind2]
            right = word[ind2:]
            
            if is_part(left):
                middle_up = capitalize(middle)
                right_up = capitalize(right)
                
                if is_part(middle_up) and is_part(right_up):
                    if not in_dic(left) and in_dic(left[:-1]) and in_dic(middle_up):
                        return (left[:-1], middle_up, right_up)
                    if not in_dic(left) and in_dic(left[:-2]) and in_dic(middle_up):
                        return (left[:-2], middle_up, right_up)  
                    if in_dic(left) and not in_dic(middle_up) and in_dic(middle_up[:-1]):
                        return (left, middle_up[:-1], right_up)
                    if in_dic(left) and not in_dic(middle_up) and in_dic(middle_up[:-2]):
                        return (left, middle_up[:-2], right_up)
                    if in_dic(left) and in_dic(middle_up):
                        return (left, middle_up, right_up)

In [12]:
s = open("C:/Users/1/Desktop/thesis/corpus_final/corpus_de.txt", "r", encoding="utf-8")

In [13]:
f = open("C:/Users/1/Desktop/thesis/corpus_final/corpus_de_splitted.txt", "w", encoding="utf-8")
w = open("C:/Users/1/Desktop/thesis/corpus_final/compounds_extracted.txt", "w", encoding="utf-8")

In [14]:
lines = s.readlines()
lines_new = []

for ind in range(len(lines)):
    line_n = lines[ind].strip("\n")
    tokens = line_n.split(" ")
    tokens_new = []
    for token in tokens:
        lem = get_lemma(token)
        
        if split(lem) and not split3(lem):
            comp = split(lem)
            tokens_new.extend([comp[0], comp[1]])
            w.write(comp[0] + "/" + comp[1] + "/" + token + "/" + str(ind) + "\n")
            
        if split3(lem) and not split(lem):
            comp = split3(lem)
            tokens_new.extend([comp[0], comp[1], comp[2]])
            w.write(comp[0] + "/" + comp[1] + "/" + comp[2] + "/" + token + "/" + str(ind) + "\n")
            
        if split(lem) and split3(lem):
            comp = split(lem)
            tokens_new.extend([comp[0], comp[1]])
            w.write(comp[0] + "/" + comp[1] + "/" + token + "/" + str(ind) + "\n")           
            
        if not split(lem) and not split3(lem):
            tokens_new.append(token)
            
    line_new = " ".join(tokens_new)
    lines_new.append(line_new)
w.close()

for line in lines_new:
    f.write(line+"\n")
f.close()